In [1]:
from functions import *
from architectures import *

%load_ext autoreload
%autoreload 2

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


## -----------------------------------------------------------------------------------------------------------------------------

In [2]:
data_lagged_features = pd.read_csv('notoptshit.csv')
data_lagged_features['Date'] = pd.to_datetime(data_lagged_features['Date'])

In [3]:
for c in tqdm(data_lagged_features.drop(['Date', 'Demand', 'Store_id', 'SKU_id'], axis=1).columns):
    a = data_lagged_features[[c, 'Store_id', 'SKU_id', 'Date']].groupby(['Store_id', 'SKU_id']).apply(lambda x : x.set_index('Date').rolling('5D', min_periods=1).mean())[[c]].reset_index()
    merged = data_lagged_features.merge(a, on=['Store_id', 'SKU_id', 'Date'])
    data_lagged_features.loc[data_lagged_features[c].isna(), c] = merged[data_lagged_features[c].isna()][c+'_y']

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:18<00:00,  3.96it/s]


In [4]:
from tqdm.contrib.itertools import product

for c, store, sku in product(data_lagged_features.drop(['Date', 'Demand'], axis=1).columns,
                      data_lagged_features['Store_id'].unique(), data_lagged_features['SKU_id'].unique()):
    cond = (data_lagged_features['SKU_id'] == sku) & (data_lagged_features['Store_id'] == store)
    data_lagged_features.loc[cond, c] = data_lagged_features[cond][c].fillna(data_lagged_features[cond][c].mean())

  0%|          | 0/16280 [00:00<?, ?it/s]

In [5]:
for c in data_lagged_features.drop(['Date', 'Demand'], axis=1).columns:
    for sku in data_lagged_features['SKU_id'].unique():
        cond = (data_lagged_features['SKU_id'] == sku)
        data_lagged_features.loc[cond, c] = data_lagged_features[cond][c].fillna(data_lagged_features[cond][c].mean())

In [294]:
df_train, df_test, target_mean, target_stdev, target, features = preprocess(data_lagged_features, scl=0)

In [295]:
set_random_seed(7)

In [296]:
df_train = df_train.reset_index()
df_test = df_test.reset_index()

df_train["time_idx"] = df_train['Date'].dt.dayofyear + (df_train['Date'].dt.year - 2015) * 365
df_test["time_idx"] = df_test['Date'].dt.dayofyear + (df_test['Date'].dt.year - 2015) * 365
min_time_idx = df_train["time_idx"].min()
df_train["time_idx"] -= min_time_idx
df_test["time_idx"] -= min_time_idx

In [297]:
df_train['Store_id'] = df_train['Store_id'].astype(str)
df_train['SKU_id'] = df_train['SKU_id'].astype(str)
df_train['Promo'] = df_train['Promo'].astype(str)
df_train['weekday'] = df_train['weekday'].astype(str)
df_train['monthday'] = df_train['monthday'].astype(str)

df_test['Store_id'] = df_test['Store_id'].astype(str)
df_test['SKU_id'] = df_test['SKU_id'].astype(str)
df_test['Promo'] = df_test['Promo'].astype(str)
df_test['weekday'] = df_test['weekday'].astype(str)
df_test['monthday'] = df_test['monthday'].astype(str)

In [298]:
#new_stores = df_test[~df_test['Store_id'].isin(df_train['Store_id'])]['Store_id'].unique()
#df_test.loc[df_test['Store_id'].isin(new_stores), "Store_id"] = "99"
#df_test = df_test.drop(df_test[df_test['Store_id'].isin(new_stores)].index)

In [299]:
df_test['Demand'] = 0

In [335]:
from pytorch_forecasting.data.encoders import NaNLabelEncoder

from pytorch_forecasting.data import (
    TimeSeriesDataSet,
    GroupNormalizer
)
max_prediction_length = 28
max_encoder_length = 60

training = TimeSeriesDataSet(
    df_train,
    time_idx="time_idx",
    target="Demand",
    group_ids=["Store_id", "SKU_id"],
    min_encoder_length=0,  # allow predictions without history
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["Store_id", "SKU_id"],
    static_reals=[],
    time_varying_known_categoricals=["Promo", "weekday", "monthday"],
    variable_groups={},
    time_varying_known_reals=[
        "time_idx",
        "Regular_Price",
        "Actual_Price",
        "Promo_percent"
    ] + df_train.columns[df_train.columns.str.contains('lag')].values.tolist(),
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[],
    target_normalizer=GroupNormalizer(
        groups=["Store_id", "SKU_id"]
    ),
    categorical_encoders={'Store_id': NaNLabelEncoder(add_nan=True)},
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

batch_size = 64
train_dataloader = training.to_dataloader(
    train=True, batch_size=batch_size, num_workers=0
)

In [385]:
import pytorch_lightning as pl
from pytorch_forecasting.metrics import QuantileLoss, SMAPE
from pytorch_forecasting.models import TemporalFusionTransformer


trainer = pl.Trainer(
    max_epochs=2,
    gpus=1,
    gradient_clip_val=0.1,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=4e-3,
    hidden_size=64,  # biggest influence network size
    attention_head_size=1,
    dropout=0.25,
    hidden_continuous_size=8,
    #output_size=7,  # QuantileLoss has 7 quantiles by default
    loss=SMAPE(),
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [386]:
trainer.fit(
    tft,
    train_dataloader=train_dataloader,
)

C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:735: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | SMAPE                           | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 2.7 K 
3  | prescalers                         | ModuleDict                      | 1.2 K 
4  | static_variable_selection          | VariableSelectionNetwork        | 5.6 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 167 K 
6  

Training: 0it [00:00, ?it/s]

In [410]:
trainer = pl.Trainer(
    max_epochs=1,
    gpus=1,
    gradient_clip_val=0.1,
)

trainer.fit(
    tft,
    train_dataloader=train_dataloader,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:735: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | SMAPE                           | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 2.7 K 
3  | prescalers                         | ModuleDict                      | 1.2 K 
4  | static_variable_selection          | VariableSelectionN

Training: 0it [00:00, ?it/s]

In [411]:
pred, ind = tft.predict(df_test, return_index=True)

In [412]:
res_test = pd.read_csv('test_kaggle.csv')
res_test['Date'] = pd.to_datetime(res_test['Date'])
res_test['pred'] = None
res_test.head()

,Store_id,SKU_id,Date,Promo,Demand,Regular_Price,Promo_Price,pred
0,1,1,2016-05-23,1.0,NaN,128.98,119.6,None
1,1,1,2016-05-24,NaN,NaN,128.98,NaN,None
2,1,1,2016-05-25,NaN,NaN,131.70,NaN,None
3,1,1,2016-05-26,NaN,NaN,131.70,NaN,None
4,1,1,2016-05-27,NaN,NaN,131.70,NaN,None


In [413]:
for i in range(pred.shape[0]):
    store, sku = int(ind.iloc[i]['Store_id']), int(ind.iloc[i]['SKU_id'])
    res_test.loc[(res_test['Store_id'] == store) & (res_test['SKU_id'] == sku), 'pred'] = pred[i].numpy()

In [414]:
ans_68 = pd.read_csv('68.csv')
ans_68

,id,Demand
0,0,661.365900
1,1,150.649200
2,2,133.698240
3,3,142.240630
4,4,136.909260
...,...,...
5965,5965,3.979614
5966,5966,6.695038
5967,5967,6.142532
5968,5968,2.497055


In [415]:
res_test.loc[res_test['pred'].isna(), 'pred'] = ans_68[res_test['pred'].isna()]['Demand'].values

In [416]:
res_test = res_test[['pred']].rename(columns={'pred' : 'Demand'})
res_test.to_csv('ans.csv', index_label='id')

In [417]:
res_test

,Demand
0,698.016968
1,95.544807
2,52.532536
3,72.302238
4,76.542
...,...
5965,3.979614
5966,6.695038
5967,6.142532
5968,2.497055
